In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import os

In [3]:
pwd = os.getcwd()
#Load main data
df = pd.read_csv(pwd+'\\data\\raw\\housing\\18100205.csv',header = 0)
#Get table of metadata
df_md = pd.read_csv(pwd+'\\data\\raw\\housing\\18100205_MetaData.csv',header = 0, skiprows =7,nrows = 43)

In [4]:
#Change REF_DATE into yyyy-mm date type
df['Month'] = pd.to_datetime(df.REF_DATE)
#Get data for all Canada
df_Can = df[(df.GEO == "Canada") ]
#Condition to get total houe value: df["New housing price indexes"].str.contains('Total')
#Get provinces name from metadata file
unique_province_id = df_md['Parent Member ID'].unique()
provinces_names = df_md[(df_md['Member ID'].isin(unique_province_id)) & (df_md['Dimension ID']==1) \
    & ~(df_md['Member ID'].isin([1,2,28]))]['Member Name']
#Get a dictionary province_df to store each province data where keys are name of provinces, values are dictionary
province_df = {}
for name in provinces_names:
    province_df[name] = df[df.GEO == name]